## Importação de bibliotecas e dados

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
data = pd.read_table("./smsspamcollection/SMSSpamCollection", sep="\t", header=None, names=["label", "mensagem"])
data.head()

,label,mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Data Pré-Processamento

Agora que temos uma compreensão básica de como é nosso conjunto de dados, vamos converter nossos rótulos em variáveis binárias, 0 para representar 'ham' (ou seja, não spam) e 1 para representar 'spam' para facilitar o cálculo.

Você pode estar se perguntando por que precisamos fazer esta etapa? A resposta para isso está em como o scikit-learn lida com as entradas. Scikit-learn lida apenas com valores numéricos e, portanto, se deixássemos nossos valores de rótulo como strings, scikit-learn faria a conversão internamente (mais especificamente, os rótulos de string serão convertidos em valores flutuantes desconhecidos).

Nosso modelo ainda seria capaz de fazer previsões se deixássemos nossos rótulos como strings, mas poderíamos ter problemas mais tarde ao calcular as métricas de desempenho, por exemplo, ao calcular nossas pontuações de precisão e recall. Portanto, para evitar 'pegadinhas' inesperadas mais tarde, é uma boa prática fazer com que nossos valores categóricos sejam alimentados em nosso modelo como números inteiros.

In [4]:
data["label"] = data.label.map({"ham":0, "spam":1})
print(data.shape)
data.head()

(5572, 2)


,label,mensagem
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Bag of Words

O que temos aqui em nosso conjunto de dados é uma grande coleção de dados de texto (5.572 linhas de dados). A maioria dos algoritmos de ML depende de dados numéricos para serem inseridos neles como entrada, e as mensagens de e-mail/sms geralmente contêm muito texto.

Aqui nós gostaríamos de apresentar o conceito Bag of Words (BoW) que é um termo usado para especificar os problemas que têm um 'bag of words' ou uma coleção de dados de texto que precisam ser trabalhados. A ideia básica do BoW é pegar um pedaço de texto e contar a frequência das palavras nesse texto. É importante notar que o conceito BoW trata cada palavra individualmente e a ordem em que as palavras ocorrem não importa.

Usando um processo pelo qual passaremos agora, podemos converter uma coleção de documentos em uma matriz, com cada documento sendo uma linha e cada palavra(token) sendo a coluna, e os valores correspondentes (linha,coluna) sendo a frequência de ocorrência de cada palavra ou token nesse documento.

## Dividindo os dados em treino e teste

Nosso primeiro passo é dividir nosso conjunto de dados em um conjunto de treinamento e teste para que possamos testar nosso modelo posteriormente.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data["mensagem"], data["label"])
print('Número de linhas no conjunto total: {}'.format(data.shape[0]))
print('Número de linhas no conjunto de treinamento: {}'.format(X_train.shape[0]))
print('Número de linhas no conjunto de teste: {}'.format(X_test.shape[0]))

Número de linhas no conjunto total: 5572
Número de linhas no conjunto de treinamento: 4179
Número de linhas no conjunto de teste: 1393


Aplicaremos o Bag os Words e converteremos os nossos dados para o formato de matriz. Para fazer isso, usaremos CountVectorizer().Há duas etapas a serem consideradas aqui:

- Em primeiro lugar, temos que ajustar nossos dados de treinamento (X_train) em CountVectorizer() e retornar a matriz.
- Em segundo lugar, temos que transformar nossos dados de teste (X_test) para retornar a matriz.

Observe que X_train são nossos dados de treinamento para a coluna 'mensagem' em nosso conjunto de dados e usaremos isso para treinar nosso modelo.
X_test são nossos dados de teste para a coluna 'mensagem' e esses são os dados que usaremos (após a transformação em uma matriz) para fazer previsões. Em seguida, compararemos essas previsões com y_test em uma etapa posterior.

In [12]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

## Implementação Naive Bayes usando scikit-learn

Utilizaremos o método sklearn.naive_bayes em nosso conjunto de dados.

Especificamente, usaremos a implementação multinomial Naive Bayes. Este classificador específico é adequado para classificação com características discretas (como no nosso caso, contagem de palavras para classificação de texto). Ele recebe contagens de palavras inteiras como sua entrada. Por outro lado, o Gaussian Naive Bayes é mais adequado para dados contínuos, pois assume que os dados de entrada têm uma distribuição Gaussiana (normal).

In [15]:
nb = MultinomialNB()
nb.fit(training_data, y_train)

MultinomialNB()

In [16]:
pred = nb.predict(testing_data)

Agora que as previsões foram feitas em nosso conjunto de teste, precisamos verificar a precisão de nossas previsões.

## Avaliando nosso modelo

Para problemas de classificação que são distorcidos em suas distribuições de classificação, como no nosso caso, por exemplo, se tivéssemos 100 mensagens de texto e apenas 2 fossem spam e as outras 98 não fossem, a precisão por si só não é uma métrica muito boa. Poderíamos classificar 90 mensagens como não spam (incluindo as 2 que eram spam, mas as classificamos como não spam, portanto, seriam falsos negativos) e 10 como spam (todos os 10 falsos positivos) e ainda obter uma pontuação de precisão razoavelmente boa. Para esses casos, precisão e recall são muito úteis. Essas duas métricas podem ser combinadas para obter a pontuação F1, que é a média ponderada das pontuações de precisão e recall. Essa pontuação pode variar de 0 a 1, sendo 1 a melhor pontuação possível na F1.

Usaremos todas as 4 métricas para garantir que nosso modelo funcione bem. Para todas as 4 métricas cujos valores podem variar de 0 a 1, ter uma pontuação o mais próxima possível de 1 é um bom indicador do desempenho do nosso modelo.

In [18]:
print('Acurácia: ', format(accuracy_score(y_test, pred)))
print('Precisão: ', format(precision_score(y_test, pred)))
print('Recall: ', format(recall_score(y_test, pred)))
print('F1 score: ', format(f1_score(y_test, pred)))

Acurácia:  0.9856424982053122
Precisão:  0.9479768786127167
Recall:  0.9371428571428572
F1 score:  0.942528735632184


## Conclusão

Uma das principais vantagens que o Naive Bayes tem sobre outros algoritmos de classificação é sua capacidade de lidar com um número extremamente grande de recursos. No nosso caso, cada palavra é tratada como um recurso e existem milhares de palavras diferentes. Além disso, ele funciona bem mesmo com a presença de recursos irrelevantes e é relativamente inalterado por eles. A outra grande vantagem que tem é a sua relativa simplicidade. O Naive Bayes funciona bem imediatamente e o ajuste de seus parâmetros raramente é necessário, exceto geralmente nos casos em que a distribuição dos dados é conhecida. Raramente supera os dados. Outra vantagem importante é que os tempos de treinamento e previsão do modelo são muito rápidos para a quantidade de dados que ele pode manipular. Em suma, Naive Bayes é realmente uma jóia de algoritmo!